In [294]:
#establishing environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy
from scipy.stats import pearsonr
from scipy import signal as sig
from scipy.io import loadmat, savemat

In [295]:
split_1 = loadmat('/Users/carlosaguila/PycharmProjects/CNT_Interictal_Spikes/matlab_import/split_1.mat')

In [296]:
values_gdf0 = split_1['values_all'][0,0] #30721 x 146
values_gdf1 = split_1['values_all'][0,1]
values_gdf2 = split_1['values_all'][0,2]
values_gdf3 = split_1['values_all'][0,3]
values_gdf4 = split_1['values_all'][0,4]
values_gdf5 = split_1['values_all'][0,5]

seq_0 = split_1['seqs_all'][0,0] #[0,0][:,0] - gives you first seq in gdf 0, channel info, [0,0][:,1] - gives you spike index
seq_1 = split_1['seqs_all'][0,1]
seq_2 = split_1['seqs_all'][0,2]
seq_3 = split_1['seqs_all'][0,3]
seq_4 = split_1['seqs_all'][0,4]
seq_5 = split_1['seqs_all'][0,5]

ch_labels0 = split_1['ch_labels_all'][0,0]
fs = 512;

In [297]:
np.shape(split_1['values_all'])

(1, 6)

In [298]:
#NOTES:
#LOOK INTO MAYBE REMOVING NAN'S BUT THAT WOULD SHIFT THE CHLABELS. 
#PLOT CODE IN MATLAB DOESN'T CHANGE THE NAN'S IT JUST DOESN'T OFFSET THEM. 

In [299]:
print(np.shape(values_gdf0))

(30721, 146)


In [364]:
#function to find the mean_max in a sequence split(gdf)
def mean_max(sequence_split, values):
    seq_0_concat = np.concatenate(sequence_split)
    seq_0_concat = np.concatenate(seq_0_concat) #double concatenate so that everything is in a array corresponding - basically turns into GDF
    ch_uniq = np.unique(seq_0_concat[:,0]) #finds the unique channels in each run_time
    #finds the mean of the max of each channel's spike index.
    meanmax_per_ch = [];
    all_max = []
    for ch in ch_uniq:
        x = np.where(seq_0_concat[:,0] == ch)[0] #index where all spikes are per channel per run_time
        max_in_ch = []
        chs = []
        for i in x:
            val = values[seq_0_concat[i,1]-20:seq_0_concat[i,1]+20, ch] #finding value at the spike and channel from x
            val_max = np.max(np.abs(val))
            max_in_ch.append(val_max)
            chs.append(np.unique(ch))
        meanmax_per_ch.append(np.mean(max_in_ch))
        all_max.append([np.concatenate(chs),max_in_ch])
        
    chs2 = (np.concatenate(np.transpose(all_max)[0]))
    maxs2 = (np.concatenate(np.transpose(all_max)[1]))
    all_max_2 = [chs2,maxs2] #reshape of maxs in all channels.
    return meanmax_per_ch, ch_uniq, all_max_2

In [366]:
mean_max_seq_0, ch_uniq, all_max = mean_max(seq_0,values_gdf0)
print(np.shape(all_max))

(2, 523)


In [408]:
#creates a concatenation of ALL gdf's mean absolute max value in every channel
def ALL_mean_max(split_1): #input would be the complete matrix assuming 'values_all' and 'seqs_all' are the base names
    mean_max_ALL = []
    ch_uniq_ALL = []
    max_I_ALL = []
    
    for I in range(len(split_1['values_all'][0])):
        values_gdf_I = split_1['values_all'][0,I]
        seq_I = split_1['seqs_all'][0,I]
        mean_max_seq_I, ch_uniq_I, all_max_I = mean_max(seq_I,values_gdf_I)
        mean_max_ALL.append(mean_max_seq_I)
        ch_uniq_ALL.append(ch_uniq_I)
        max_I_ALL.append(all_max_I)
    
    ch_uniq_AL_C = np.concatenate(ch_uniq_ALL)
    mean_max_ALL_C = np.concatenate(mean_max_ALL)
    ALL_CH = []
    ALL_maxvalues = []
    for s in range(len(max_I_ALL)):
        ALL_CH.append((max_I_ALL[s][0]))
        ALL_maxvalues.append((max_I_ALL[s][1]))
    ALL_CH = np.concatenate(ALL_CH)
    ALL_maxvalues = np.concatenate(ALL_maxvalues)
    max_I_ALL = [ALL_CH,ALL_maxvalues]
    ALL_mean_max_with_ch = [ch_uniq_AL_C,mean_max_ALL_C]
    return ALL_mean_max_with_ch, max_I_ALL

In [411]:
all_mean_max, max_I_ALL_C = ALL_mean_max(split_1)
np.shape(max_I_ALL_C)

(2, 3390)

In [446]:
#creates a compilation of every channel in all gdfs and there respective means of max absolute peak values 
def meanofmeanmax_per_ch(split_1): #input is the complete split file.
    all_mean_max, max_I_all = ALL_mean_max(split_1) #uses ALL_mean_max function to get you a complete list of concatenated mean max's

    #code to get means of means per channel.
    all_mean_max = np.transpose(all_mean_max)
    max_I_all = np.transpose(max_I_all)
    popmean= np.nanmean(max_I_all[:,1])
    popstd = np.nanstd(max_I_all[:,1])
    ch_uniq = np.unique(all_mean_max[:,0]) #finds the unique channels in concatenated list
    ch_uniq_ALL = np.unique(max_I_all[:,0])
    
    means = []
    means_from_all_maxes = []
    std_from_all_maxes = []
    for ch in ch_uniq:
        x = np.where(all_mean_max[:,0] == ch)[0]#index where all spikes are per channel per run_time
        means.append(np.mean(all_mean_max[x,1]))
    for ch2 in ch_uniq_ALL:
        x2 = np.where(max_I_all[:,0] == ch2)[0]
        means_from_all_maxes.append(np.mean(max_I_all[x2,1]))
        std_from_all_maxes.append(np.std(max_I_all[x2,1]))
    meanofmeanmax = [ch_uniq,means]    
    stats_per_ch = [ch_uniq_ALL, means_from_all_maxes, std_from_all_maxes]
    
    return np.transpose(meanofmeanmax), np.transpose(stats_per_ch), popmean, popstd # ['channel','mean of mean max'] ['channel', 'mean of maxes for all channels', 'std of maxes for all channels']

In [447]:
meanmax_ch_ALL, stats_ch_ALL, popmean, popstd = meanofmeanmax_per_ch(split_1)

In [452]:
print(np.shape(meanmax_ch_ALL))
print(np.shape(stats_ch_ALL))
print('"population" mean:', popmean)
print('"population" std:', popstd)
print(len(stats_ch_ALL))

(107, 2)
(107, 3)
"population" mean: 205.21694186524024
"population" std: 151.0407334371281
107


In [472]:
#test for significance
from scipy import stats as st

#perform 1-sample t test
def stu_ttest_per_chn(split_1):
    _ , max_I_all = ALL_mean_max(split_1)
    _ , stats_ch_ALL, popmean, popstd = meanofmeanmax_per_ch(split_1)
    max_I_all = np.transpose(max_I_all)
    ch_uniq_ALL = np.unique(max_I_all[:,0])
    stats_per_chn = []
    for ch2 in ch_uniq_ALL:
        x2 = np.where(max_I_all[:,0] == ch2)[0]
        stats = st.ttest_1samp(a=max_I_all[x2,1], popmean=popmean)
        stats_per_chn.append(stats)
    stats_per_chn_labeled = [ch_uniq_ALL, stats_per_chn];
    return np.transpose(stats_per_chn_labeled)

In [473]:
x = stu_ttest_per_chn(split_1)


In [476]:
x

array([[7.0,
        Ttest_1sampResult(statistic=6.549282358995786, pvalue=1.292548370549977e-05)],
       [8.0,
        Ttest_1sampResult(statistic=-1.006259307640261, pvalue=0.32839977535010534)],
       [9.0,
        Ttest_1sampResult(statistic=-3.1379454223491385, pvalue=0.025725861574554375)],
       [11.0,
        Ttest_1sampResult(statistic=-1.3444369058008605, pvalue=0.22740510003413508)],
       [12.0,
        Ttest_1sampResult(statistic=-7.224399073459904, pvalue=0.00035667054355398363)],
       [15.0, Ttest_1sampResult(statistic=nan, pvalue=nan)],
       [16.0,
        Ttest_1sampResult(statistic=-6.168680954143606, pvalue=0.0001649629744595394)],
       [17.0,
        Ttest_1sampResult(statistic=-26.38827563953052, pvalue=2.8733080324990944e-08)],
       [28.0,
        Ttest_1sampResult(statistic=1.2392141775534729, pvalue=0.2615394104927984)],
       [29.0,
        Ttest_1sampResult(statistic=-9.305934262930185, pvalue=0.0026267906697071824)],
       [30.0,
        Ttest_1